In [1]:
%config InlineBackend.figure_format = 'retina'
%load_ext watermark
%watermark

Last updated: 2024-06-27T17:53:38.437190-04:00

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.22.2

Compiler    : MSC v.1937 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 167 Stepping 1, GenuineIntel
CPU cores   : 16
Architecture: 64bit



In [2]:
import geopandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import xgboost as xgb

pd.set_option('display.float_format', lambda x: '%.3f' % x)

%matplotlib inline
%watermark -w
%watermark -iv

Watermark: 2.4.3

seaborn   : 0.13.2
geopandas : 0.14.3
numpy     : 1.26.4
pandas    : 2.2.1
matplotlib: 3.8.3
xgboost   : 2.1.0



In [3]:
sns.set_style('ticks')

warnings.filterwarnings('ignore')


seed = 1234
np.random.seed(seed)
random.seed(seed)

## Funcoes de Apoio

**IMPORTANTE**: Por ser uma analise pontual e para fins pessoais, optei por deixar o codigo do do tratamento nas variaveis de treino e de teste na mesma celula.

In [4]:
# Separar os dados em treino e em teste

def selecao_teste_treino(dados, test_ratio):
    indices_aleat = np.random.permutation(len(dados))
    tamanho_base = int(len(dados) * test_ratio)
    indices_test = indices_aleat[:tamanho_base]
    indices_treino = indices_aleat[tamanho_base:]
    return dados.iloc[indices_treino], dados.iloc[indices_test]

In [5]:
# AJUSTE DAS COLUNAS SITUACAO DE CONCLUSAO E ANO DE CONCLUSAO

def ajuste_var_ano_conclusao(linha, coluna1, coluna2):
    """
    Ajusta a variável de ano de conclusão com base nas colunas especificadas.

    Parâmetros:
    dados (DataFrame): O DataFrame contendo os dados.
    coluna1 (str): O nome da primeira coluna para verificar.
    coluna2 (str): O nome da segunda coluna para verificar.

    Retorna:
    int: O ano ajustado de conclusão.
    """

    mapping_coluna2 = {
        1: 2022,
        2: 2021,
        3: 2020,
        4: 2019,
        5: 2018,
        6: 2017,
        7: 2016,
        8: 2015,
        9: 2014,
        10: 2013,
        11: 2012,
        12: 2011,
        13: 2010,
        14: 2009,
        15: 2008,
        16: 2007,
        17: 2006
    }



    if linha[coluna2] == 0 and linha[coluna1] == 2:
        return 2023
    elif linha[coluna2] == 0 and linha[coluna1] == 3:
        return 2024
    elif linha[coluna2] == 0 and linha[coluna1] == 4:
        return 2025 # são apenas 10.517 casos
    elif linha[coluna2] == 0 and linha[coluna1] == 1:
        return 2006 # são 122.901 casos de pessoas que já concluíram e não informaram o ano
    elif linha[coluna2] != 0:
        return mapping_coluna2.get(linha[coluna2], linha[coluna2])
    else:
        return linha[coluna2]

## Dados

In [6]:
# Data

data = pd.read_csv("microdados_enem_2023/DADOS/MICRODADOS_ENEM_2023.csv", encoding='latin', sep = ';')

In [7]:
colunas1 = ['NU_INSCRICAO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
            'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU',
           'IN_TREINEIRO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT',
           ]

colunas2 = data.loc[:,'Q001':'Q025'].columns.to_list()
colunas3 = colunas1 + colunas2

dados_modelo = data[colunas3]

A colunas target foi definida considerando se o aluno faltou no primeiro dia, conforme avaliado na descritiva dos dados.

In [8]:
def faltou(coluna1):
    if coluna1 == 1 or coluna1 == 2:
        return 0 # presente
    else:
        return 1 # falta

dados_modelo['target'] = dados_modelo['TP_PRESENCA_CN'].apply(faltou)

dados_modelo.head()

,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,IN_TREINEIRO,TP_PRESENCA_CN,TP_PRESENCA_CH,...,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,target
0,210059085136,14,M,2,1,1,17,0,0,0,...,C,B,B,A,B,B,A,A,B,1
1,210059527735,12,M,2,1,1,16,0,0,0,...,A,B,B,A,A,C,A,D,B,1
2,210061103945,6,F,1,1,1,0,0,1,1,...,A,A,B,A,A,A,A,A,B,0
3,210060214087,2,F,1,3,2,0,0,1,1,...,A,A,B,A,A,D,A,A,B,0
4,210059980948,3,F,1,3,2,0,0,1,1,...,A,A,B,A,A,B,A,A,A,0


In [9]:
# Separar os dados em treino e em teste

dados_treino, dados_teste = selecao_teste_treino(dados_modelo, 0.4)

In [10]:
print(f'A base treino possui {len(dados_treino)} registros.')
print(f'A base teste possui {len(dados_teste)} registros.')
print(f'A proporcao de resgistros de treino e de {len(dados_treino)/len(dados_modelo)*100}%.')

A base treino possui 2360373 registros.
A base teste possui 1573582 registros.
A proporcao de resgistros de treino e de 60.0%.


In [11]:
# Avaliando a proporcao da variavel target dentro de cada base de dados.
# Essa e uma pratica que eu gosto de adotar quando realizo a separacao das bases, apenas como um criterio de validacao.

# print(f'A proporcao de presentes na base treino e de {round(dados_treino['target'].value_counts()[0]/dados_treino['target'].value_counts().sum()*100,5)}%.')
# print(f'A proporcao de presentes na base teste e de {round(dados_teste['target'].value_counts()[0]/dados_teste['target'].value_counts().sum()*100,5)}%.')

In [12]:
caracteristicas_participantes = dados_treino.drop('target', axis = 1)
labels = dados_treino['target'].copy()



############# AJUSTE TESTE

caracteristicas_participantes_teste = dados_teste.drop('target', axis = 1)
labels_teste = dados_teste['target'].copy()

In [13]:
# Removendo as colunas com a informacao de presenca x ausencia

caracteristicas_participantes = caracteristicas_participantes.drop(['NU_INSCRICAO','TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',], axis=1)


################ AJUSTE TESTE

caracteristicas_participantes_teste = caracteristicas_participantes_teste.drop(['NU_INSCRICAO','TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_PRESENCA_MT',], axis=1)

## Tratamentos

### Regras Adotadas:
- Criacao de uma nova variavel para: TP_ST_CONCLUSAO e TP_ANO_CONCLUIU: foi criada uma nova variavel acima contemplando as duas variaveis
- Remocao da var IN_TREINEIRO: todos os participantes inscritos como treineiros possuem a a classificacao: "Estou cursando e concluirei o Ensino Médio após 2023 em TP_ST_CONCLUSAO".
- Q005 - NUMERICA 
  
- Qualitativas Ordinais:
  - TP_FAIXA_ETARIA (ja codificado na origem)
  - Q001 e Q002
  - Q003 e Q004 (sera considerada ordinal pois e possivel extrair uma hierarquia nas profissoes)
  - Q006
  - Q007
  - Q008
  - Q009
  - Q010
  - Q011
  - Q012
  - Q013
  - Q014
  - Q015
  - Q016
  - Q017
  - Q019
  - Q022
  - Q024
  
- Qualitativas: 
  - BINARIA - 2 VAR:
    - TP_SEXO (considerar 0 e 1)
    - Q018 (considerar 0 e 1)
    - Q020 (considerar 0 e 1)
    - Q021 (considerar 0 e 1)
    - Q023 (considerar 0 e 1)
    - Q025 (considerar 0 e 1)
  
  - \+2 VARS - NOMINAL:
    - TP_ESTADO_CIVIL
    - TP_COR_RACA
 


- Pensando:
  - Considerar casas com mais de 10 pessoas morando como 10.

### - CRIACAO DA VARIAVEL ANO_CONCLUSAO_AJUSTADO

In [14]:
# Ajuste das variaveis ano e data de conclusao

caracteristicas_participantes['ANO_CONCLUSAO_AJUSTADO'] = caracteristicas_participantes.apply(ajuste_var_ano_conclusao, axis=1,
                                                                                   coluna1 = 'TP_ST_CONCLUSAO', coluna2 = 'TP_ANO_CONCLUIU')



########################### AJUSTE TESTE

# Ajuste das variaveis ano e data de conclusao

caracteristicas_participantes_teste['ANO_CONCLUSAO_AJUSTADO'] = caracteristicas_participantes_teste.apply(ajuste_var_ano_conclusao, axis=1,
                                                                                   coluna1 = 'TP_ST_CONCLUSAO', coluna2 = 'TP_ANO_CONCLUIU')

In [15]:
# Remocao das variaveis:
# TP_ST_CONCLUSAO e TP_ANO_CONCLUIU: foi criada uma nova variavel acima contemplando as duas variaveis
# IN_TREINEIRO: todos os participantes inscritos como treineiros possuem a a classificacao: Estou cursando e concluirei o Ensino Médio após 2023 em TP_ST_CONCLUSAO

caracteristicas_participantes = caracteristicas_participantes.drop(['TP_ANO_CONCLUIU', 'IN_TREINEIRO', 'TP_ST_CONCLUSAO'], axis=1)


############ AJUSTE TESTE

caracteristicas_participantes_teste = caracteristicas_participantes_teste.drop(['TP_ANO_CONCLUIU', 'IN_TREINEIRO', 'TP_ST_CONCLUSAO'], axis=1)

## - Variaveis Categoricas

In [16]:
# Listando as colunas categoricas por tipo 

cat_cols_ord = ['TP_FAIXA_ETARIA', 'Q001', 'Q002', 'Q003', 'Q004', 'Q006', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012',
                'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q019', 'Q022', 'Q024']
cat_quali_2_faixas = ['TP_SEXO', 'Q018', 'Q020', 'Q021', 'Q023','Q025']
cat_quali_2_mais_faixas = ['TP_ESTADO_CIVIL', 'TP_COR_RACA']


In [17]:
# Definindo a ordem das categorias

categories_order = {
    'TP_FAIXA_ETARIA': list(range(1, 21)),
    'TP_SEXO' : ['M', 'F'],
    'Q001': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
    'Q002': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
    'Q003': ['A', 'B', 'C', 'D', 'E', 'F'],
    'Q004': ['A', 'B', 'C', 'D', 'E', 'F'],
    'Q006': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q'],
    'Q007': ['A', 'B', 'C', 'D'],
    'Q008': ['A', 'B', 'C', 'D', 'E'],
    'Q009': ['A', 'B', 'C', 'D', 'E'],
    'Q010': ['A', 'B', 'C', 'D', 'E'],
    'Q011': ['A', 'B', 'C', 'D', 'E'],
    'Q012': ['A', 'B', 'C', 'D', 'E'],
    'Q013': ['A', 'B', 'C', 'D', 'E'],
    'Q014': ['A', 'B', 'C', 'D', 'E'],
    'Q015': ['A', 'B', 'C', 'D', 'E'],
    'Q016': ['A', 'B', 'C', 'D', 'E'],
    'Q017': ['A', 'B', 'C', 'D', 'E'],
    'Q019': ['A', 'B', 'C', 'D', 'E'],
    'Q020': ['A', 'B'],
    'Q021': ['A', 'B'],
    'Q022': ['A', 'B', 'C', 'D', 'E'],
    'Q023': ['A', 'B'],
    'Q024': ['A', 'B', 'C', 'D', 'E']
}

In [18]:
# Variaveis Ordinais

ordinal_encoder = OrdinalEncoder(categories=[categories_order[var] for var in cat_cols_ord])

caracteristicas_participantes[cat_cols_ord] = ordinal_encoder.fit_transform(caracteristicas_participantes[cat_cols_ord])

############################### AJUSTE TESTE

caracteristicas_participantes_teste[cat_cols_ord] = ordinal_encoder.transform(caracteristicas_participantes_teste[cat_cols_ord])

In [19]:
caracteristicas_participantes

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,Q001,Q002,Q003,Q004,Q005,Q006,...,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,ANO_CONCLUSAO_AJUSTADO
2143670,1.000,F,1,1,2.000,4.000,5.000,1.000,4,6.000,...,0.000,A,2.000,B,A,4.000,A,1.000,B,2023
2983198,1.000,F,1,3,4.000,6.000,3.000,3.000,4,10.000,...,0.000,B,4.000,A,A,4.000,A,4.000,B,2023
968989,10.000,F,1,1,3.000,3.000,3.000,1.000,4,6.000,...,0.000,B,2.000,A,B,3.000,A,1.000,B,2013
1659500,2.000,M,1,3,4.000,4.000,5.000,5.000,4,1.000,...,0.000,A,1.000,A,A,2.000,A,1.000,B,2023
2097665,3.000,M,1,2,2.000,1.000,2.000,2.000,3,4.000,...,0.000,A,1.000,B,B,3.000,A,1.000,B,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978124,9.000,F,1,3,1.000,3.000,1.000,1.000,2,1.000,...,0.000,A,1.000,A,A,1.000,A,0.000,B,2025
1956917,8.000,F,1,3,7.000,4.000,5.000,1.000,3,1.000,...,0.000,A,2.000,A,A,2.000,B,1.000,B,2016
165158,1.000,F,1,1,2.000,1.000,0.000,0.000,3,3.000,...,0.000,B,2.000,A,A,3.000,A,2.000,B,2023
2548435,4.000,M,1,2,3.000,4.000,5.000,1.000,2,2.000,...,0.000,B,0.000,A,A,2.000,A,1.000,B,2021


In [20]:
# Aplicacao da codificacao
# VARIAVEIS BINARIAS


mapa_binario = {'A': 0, 'B': 1, 'M':0, 'F':1}

caracteristicas_participantes[cat_quali_2_faixas] = caracteristicas_participantes[cat_quali_2_faixas].applymap(mapa_binario.get)

###################### AJUSTE TESTE

caracteristicas_participantes_teste[cat_quali_2_faixas] = caracteristicas_participantes_teste[cat_quali_2_faixas].applymap(mapa_binario.get)


In [21]:
estado_civil_map = {
    0: 'Não informado',
    1: 'Solteiro(a)',
    2: 'Casado(a)/Mora com companheiro(a)',
    3: 'Divorciado(a)/Desquitado(a)/Separado(a)',
    4: 'Viúvo(a)'
}

cor_raca_map = {
    0: 'Nao declarado',
    1: 'Branca',
    2: 'Preta',
    3: 'Parda',
    4: 'Amarela',
    5: 'Indigena',
    6: 'Nao declarado'
}


# Aplicando o map
caracteristicas_participantes['TP_ESTADO_CIVIL'] = caracteristicas_participantes['TP_ESTADO_CIVIL'].map(estado_civil_map)
caracteristicas_participantes['TP_COR_RACA'] = caracteristicas_participantes['TP_COR_RACA'].map(cor_raca_map)



######################### AJUSTE TESTE

caracteristicas_participantes_teste['TP_ESTADO_CIVIL'] = caracteristicas_participantes_teste['TP_ESTADO_CIVIL'].map(estado_civil_map)
caracteristicas_participantes_teste['TP_COR_RACA'] = caracteristicas_participantes_teste['TP_COR_RACA'].map(cor_raca_map)


In [22]:
# Aplicacao da codificacao
# VARIAVEIS NOMINAIS

onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoded = onehot_encoder.fit_transform(caracteristicas_participantes[cat_quali_2_mais_faixas])


onehot_feature_names = onehot_encoder.get_feature_names_out()
onehot_encoded_df = pd.DataFrame(onehot_encoded, columns=onehot_feature_names)


#caracteristicas_participantes = caracteristicas_participantes.drop(columns=cat_quali_2_mais_faixas).join(onehot_encoded_df)
# teste = caracteristicas_participantes.drop(columns=cat_quali_2_mais_faixas).join(onehot_encoded_df)

# Estava gerando um erro de correspondencia e, consequentemente, gerando valores NA... Para resolver, retirei os index

caracteristicas_participantes.reset_index(drop = True, inplace=True)
onehot_encoded_df.reset_index(drop = True, inplace = True)


caracteristicas_participantes = caracteristicas_participantes.drop(columns=cat_quali_2_mais_faixas).join(onehot_encoded_df)

display(caracteristicas_participantes)




############################### AJUSTE TESTE

onehot_encoded_test = onehot_encoder.transform(caracteristicas_participantes_teste[cat_quali_2_mais_faixas])

onehot_feature_names1 = onehot_encoder.get_feature_names_out()
onehot_encoded_df1 = pd.DataFrame(onehot_encoded_test, columns=onehot_feature_names1)


#caracteristicas_participantes = caracteristicas_participantes.drop(columns=cat_quali_2_mais_faixas).join(onehot_encoded_df)
# teste = caracteristicas_participantes.drop(columns=cat_quali_2_mais_faixas).join(onehot_encoded_df)

# Estava gerando um erro de correspondencia e, consequentemente, gerando valores NA... Para resolver, retirei os index

caracteristicas_participantes_teste.reset_index(drop = True, inplace=True)
onehot_encoded_df1.reset_index(drop = True, inplace = True)


caracteristicas_participantes_teste = caracteristicas_participantes_teste.drop(columns=cat_quali_2_mais_faixas).join(onehot_encoded_df1)

display(caracteristicas_participantes_teste)


,TP_FAIXA_ETARIA,TP_SEXO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,...,TP_ESTADO_CIVIL_Divorciado(a)/Desquitado(a)/Separado(a),TP_ESTADO_CIVIL_Não informado,TP_ESTADO_CIVIL_Solteiro(a),TP_ESTADO_CIVIL_Viúvo(a),TP_COR_RACA_Amarela,TP_COR_RACA_Branca,TP_COR_RACA_Indigena,TP_COR_RACA_Nao declarado,TP_COR_RACA_Parda,TP_COR_RACA_Preta
0,1.000,1,2.000,4.000,5.000,1.000,4,6.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
1,1.000,1,4.000,6.000,3.000,3.000,4,10.000,0.000,4.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
2,10.000,1,3.000,3.000,3.000,1.000,4,6.000,0.000,2.000,...,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
3,2.000,0,4.000,4.000,5.000,5.000,4,1.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
4,3.000,0,2.000,1.000,2.000,2.000,3,4.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2360368,9.000,1,1.000,3.000,1.000,1.000,2,1.000,2.000,1.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
2360369,8.000,1,7.000,4.000,5.000,1.000,3,1.000,0.000,2.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
2360370,1.000,1,2.000,1.000,0.000,0.000,3,3.000,0.000,4.000,...,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
2360371,4.000,0,3.000,4.000,5.000,1.000,2,2.000,0.000,2.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000


,TP_FAIXA_ETARIA,TP_SEXO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,...,TP_ESTADO_CIVIL_Divorciado(a)/Desquitado(a)/Separado(a),TP_ESTADO_CIVIL_Não informado,TP_ESTADO_CIVIL_Solteiro(a),TP_ESTADO_CIVIL_Viúvo(a),TP_COR_RACA_Amarela,TP_COR_RACA_Branca,TP_COR_RACA_Indigena,TP_COR_RACA_Nao declarado,TP_COR_RACA_Parda,TP_COR_RACA_Preta
0,8.000,1,1.000,1.000,1.000,0.000,4,1.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
1,3.000,0,2.000,3.000,0.000,0.000,4,2.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
2,2.000,0,4.000,4.000,2.000,1.000,3,3.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000
3,1.000,0,6.000,5.000,4.000,4.000,4,15.000,0.000,2.000,...,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
4,7.000,1,7.000,3.000,0.000,0.000,7,1.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573577,1.000,1,5.000,4.000,3.000,2.000,5,4.000,0.000,1.000,...,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
1573578,6.000,0,4.000,4.000,1.000,1.000,5,6.000,0.000,1.000,...,0.000,0.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000
1573579,3.000,0,4.000,1.000,3.000,1.000,5,2.000,0.000,2.000,...,0.000,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
1573580,1.000,1,2.000,1.000,2.000,1.000,5,2.000,0.000,4.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000


In [23]:
## Regressao Logistica

model_log = LogisticRegression(class_weight='balanced', max_iter=1000, verbose=1)
model_log.fit(caracteristicas_participantes, labels)

LogisticRegression(class_weight='balanced', max_iter=1000, verbose=1)

In [24]:
y_pred = model_log.predict(caracteristicas_participantes_teste)

In [25]:
print(f"Model Accuracy: {model_log.score(caracteristicas_participantes_teste, labels_teste)}")

Model Accuracy: 0.663967305167446


In [26]:
print(classification_report(labels_teste, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.70      0.74   1077828
           1       0.47      0.59      0.53    495754

    accuracy                           0.66   1573582
   macro avg       0.63      0.64      0.63   1573582
weighted avg       0.69      0.66      0.67   1573582



In [27]:
# XGBoost

model_xgb = xgb.XGBClassifier(use_label_encoder = False, eval_metric = 'logloss')
model_xgb.fit(caracteristicas_participantes, labels)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [28]:
y_pred_xgb = model_xgb.predict(caracteristicas_participantes_teste)

In [29]:
print(classification_report(labels_teste, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.73      0.89      0.81   1077828
           1       0.56      0.29      0.38    495754

    accuracy                           0.70   1573582
   macro avg       0.65      0.59      0.59   1573582
weighted avg       0.68      0.70      0.67   1573582



## XGBoost - Grid

Como a base e muito grande, nao recomendo realizar os passos de Grid, o tempo para o realizar os ajustes sera muito alto.
Minha recomendacao e utilizar uma amostra menor para definir os parametros.


In [30]:
# XGBoost - Grid

# Como a base e muito grande, nao recomendo realizar os passos de Grid, o tempo para o realizar os ajustes sera muito alto.
# Minha recomendacao e utilizar uma amostra menor para definir os parametros.

model_xgb1 = xgb.XGBClassifier(use_label_encoder = False, eval_metric = 'logloss')

In [31]:
from sklearn.model_selection import GridSearchCV

######################## Ideias

Definir o grid de hiperparâmetros
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 5, 7, 10],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'n_estimators': [100, 200, 300],
    'scale_pos_weight': [1, 2, 3, 4]
}

############# Configurar a busca em grid
grid_search = GridSearchCV(estimator=model_xgb1, param_grid=param_grid, scoring='f1', n_jobs=-1, cv=3, verbose=2)


In [32]:
# grid_result = grid_search.fit(caracteristicas_participantes, labels)

### TESTES - EM CONSTRUCAO

############# Melhor combinação de hiperparâmetros
best_params = grid_result.best_params_
print("Melhores parâmetros: ", best_params)

############ Melhor pontuação do modelo
best_score = grid_result.best_score_
print("Melhor pontuação F1: ", best_score)

## Ajustando a regressao logistica usando o StatModels

import statsmodels.api as sm

caracteristicas_participantes = caracteristicas_participantes.reset_index(drop=True)
labels = labels.reset_index(drop=True)

model_log_stat = sm.Logit(caracteristicas_participantes, labels).fit()

# Obter um resumo do modelo
summary = model_log_stat.summary()
print(summary)

# Extrair os coeficientes e valores p
coefficients = model_log_stat.params
p_values = model_log_stat.pvalues

# Avaliar os coeficientes e seus valores p
significant_vars = p_values[p_values < 0.05].index
print(f"Variáveis significativas: {significant_vars}")

In [38]:
# Set a threshold (e.g., median value or any other logic)
threshold = labels.median()

# Convert target to binary values based on the threshold
labels = (labels > threshold).astype(int)

# Ensure indices of characteristics and labels are aligned
caracteristicas_participantes = caracteristicas_participantes.reset_index(drop=True)
labels = labels.reset_index(drop=True)

# Fit the logistic regression model
model_log_stat = sm.Logit(labels, caracteristicas_participantes).fit()

# Get a summary of the model
summary = model_log_stat.summary()
print(summary)


Optimization terminated successfully.
         Current function value: 0.576369
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:              2360373
Model:                          Logit   Df Residuals:                  2360335
Method:                           MLE   Df Model:                           37
Date:                Thu, 27 Jun 2024   Pseudo R-squ.:                 0.07493
Time:                        18:03:50   Log-Likelihood:            -1.3604e+06
converged:                       True   LL-Null:                   -1.4706e+06
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                              coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------
TP_FAIXA_ETARIA   